In [ ]:
# DECOMMENTARE PRIMA DELL'UTILIZZO
#!pip install psycopg2
#!pip install plotly
#!pip install wfdb

In [ ]:
import matplotlib.pyplot as plt
import random
from datetime import datetime
import plotly.graph_objects as go
import string
import pandas as pd
import psycopg2
from random import randrange
from datetime import timedelta
from psycopg2 import Error
import statistics
import numpy as np

# Connection PostgreSQL

In [ ]:
def connect_db(database):
    try:
        # Connect to an existing database
        connection = psycopg2.connect(user="postgres",
                                      password="admin",
                                      host="127.0.0.1",
                                      port="5432",
                                      database=database)

        # Create a cursor to perform database operations
        cursor = connection.cursor()

        print("PostgreSQL server information")
        #print(connection.get_dsn_parameters(), "\n")

        # Executing a SQL query
        cursor.execute("SELECT version();")

        # Fetch result
        record = cursor.fetchone()
        print("You are connected to - ", record, "\n")
        
        return (connection,cursor)

    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    
def close_connection(connection):
     if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

# Lettura dataset fetal_health

In [ ]:
dataset = pd.read_csv('fetal_health.csv', index_col=0)

In [ ]:
dataset.head()

,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,mean_value_of_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
baseline value,,,,,,,,,,,,,,,,,,,,,
120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,2.4,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,10.4,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,13.4,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,23.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,19.9,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [ ]:
dataset.shape

(2126, 21)

In [ ]:
def random_date(start, end):
    
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    
    return start + timedelta(seconds=random_second)

#randomizziamo le date e gli orari delle rilevazioni

In [ ]:
#https://physionet.org/content/sufhsdb/1.0.1/

import pandas as pd

d2 = pd.read_excel("FetalPCGSpreadsheet.xlsx")

d2.head()

,Subject ID,Number of Gravid/Alive/Abortion,Maternal BMI,Pregnancy Term (weeks),"Fetus gender (B: Boy, G: Girl)",Mother’s age (years),Clinical History,"CTG Heart-rate (BPM). Each number corresponds to the average fetal heart-rate over 10 seconds of the signal, whenever available. Brackets denote unreported values",
0,F93001,NaN,28.5,32.0,B,20.0,NaN,132-148-150-150,NaN
1,F93002,2/0/1,29.7,40.0,B,23.0,NaN,150-147-152-150-148-150,NaN
2,F93003,1/0/0,32.9,37.0,B,38.0,NaN,[]-142-130-135-136,NaN
3,F93004,1/0/0,27.7,38.0,B,23.0,NaN,[]-152-144-142-143-138-135-131,NaN
4,F93005,1/0/0,21.5,26.0,B,22.0,Preterm rupture of membeans (PROM),150-156-156-154-158-155,NaN


In [ ]:
selected_rows = d2[~d2.iloc[:, 7].isnull()]

selected_rows

,Subject ID,Number of Gravid/Alive/Abortion,Maternal BMI,Pregnancy Term (weeks),"Fetus gender (B: Boy, G: Girl)",Mother’s age (years),Clinical History,"CTG Heart-rate (BPM). Each number corresponds to the average fetal heart-rate over 10 seconds of the signal, whenever available. Brackets denote unreported values",
0,F93001,NaN,28.5,32.0,B,20.0,NaN,132-148-150-150,NaN
1,F93002,2/0/1,29.7,40.0,B,23.0,NaN,150-147-152-150-148-150,NaN
2,F93003,1/0/0,32.9,37.0,B,38.0,NaN,[]-142-130-135-136,NaN
3,F93004,1/0/0,27.7,38.0,B,23.0,NaN,[]-152-144-142-143-138-135-131,NaN
4,F93005,1/0/0,21.5,26.0,B,22.0,Preterm rupture of membeans (PROM),150-156-156-154-158-155,NaN
...,...,...,...,...,...,...,...,...,...
127,F93108,3/1/1,36.9,38.0,B,30.0,NaN,150-152-144-138-133-145-138-138-141,NaN
128,F93109,2/1/0,28.3,39.0,B,33.0,Mild mitral prolapse of mother heart,180-168-183-171-177-162-159-173-176,NaN
129,F93110,2/1/0,37.9,36.0,B,30.0,NaN,142-153-144-147-147-138-145-146-148,NaN
130,F93111,3/2/0,35.0,40.0,B,33.0,NaN,143-138-132-126-129-126-134-120-124,NaN


In [ ]:
selected_rows.iloc[:,7:8]

,"CTG Heart-rate (BPM). Each number corresponds to the average fetal heart-rate over 10 seconds of the signal, whenever available. Brackets denote unreported values"
0,132-148-150-150
1,150-147-152-150-148-150
2,[]-142-130-135-136
3,[]-152-144-142-143-138-135-131
4,150-156-156-154-158-155
...,...
127,150-152-144-138-133-145-138-138-141
128,180-168-183-171-177-162-159-173-176
129,142-153-144-147-147-138-145-146-148
130,143-138-132-126-129-126-134-120-124


# CREAZIONE INSERIMENTI SESSIONE_RILEVAZIONE, ELABORA_STREAM E DATO

In [ ]:
xls = pd.ExcelFile('CTG.xlsx')
df1 = pd.read_excel(xls, 'Raw Data')

In [ ]:
nan_value = float("NaN")
df1 = df1.dropna()

In [ ]:
# CREAZIONE SESSIONE_RILEVAZIONE, ELABORA_STREAM

def create_sessione_txt(data,dataset):
    f = open("dml_insert.txt", "w")
    #elabora_str = open("elabora_stream.txt", "w")
    s = ""
    cont = 0 
    dates = list()
    for row in data.iterrows():
        cont += 1
        insert = f"INSERT INTO sessione_rilevazione (tipo,data_ora,unita_misura,nome_file,frequenza,codice_dispositivo) VALUES ('CTG','{row[1][1]}','bpm','{row[1][0]}','2 cycles/pm', '{random.randint(1, 3)}');"
        f.write(insert)
        f.write(f"INSERT INTO elabora_stream (id_dato, id_sessione_rilevazione, data_ora) VALUES ({cont},{cont},'{row[1][1]}');")
        dates.append(row[1][1])
    #elabora_str.close()
    f.close()
    dati = 0
    dato = open("dml_insert.txt", "w")
    for row in dataset.iterrows():
            
            d1 = datetime.strptime('1/1/2021 1:30 PM', '%m/%d/%Y %I:%M %p')
            d2 = datetime.strptime('10/1/2021 4:50 AM', '%m/%d/%Y %I:%M %p')
            data_h = random_date(d1, d2)
            
            #insert = f"INSERT INTO dato (id,data_ora,tipo,descrizione,valore,nome,modalita) VALUES ({cont+1},'{dates[cont]}','accelerazione', 'Numero di accellerazioni per secondo',"
            #complete = insert+str(row[1][0])+f",'accelerazione','{random.randint(1, 3)}');"
            #dato.write(complete)
            
            insert = f"INSERT INTO dato (id,data_ora,tipo,descrizione,valore,nome,modalita) VALUES ({dati+1},'{dates[dati]}','movimento', 'Numero di movimenti del feto per secondo',"
            complete = insert+str(row[1][1])+f",'movimento_fetale','{random.randint(1, 3)}');"
            dato.write(complete)
            dati += 1
            
    dato.close()
    return dati

In [ ]:
numero_dati = create_sessione_txt(df1,dataset)

# CREAZIONE INSERIMENTI RILEVAZIONE, ELABORA E DATO

In [ ]:
weight = pd.read_csv('baby_weight.csv')

weight.head()

,Unnamed: 0,Birth.Weight,Gestational.Days,Maternal.Age,Maternal.Height,Maternal.Pregnancy.Weight,Maternal.Smoker
0,1,120,284,27,62,100,False
1,2,113,282,33,64,135,False
2,3,128,279,28,64,115,True
3,4,108,282,23,67,125,True
4,5,136,286,25,62,93,False


In [ ]:
# CREAZIONE RILEVAZIONE

def create_rilevazione_txt(data):
    f = open("dml_insert.txt", "w")
    for row in data.iterrows():
        d1 = datetime.strptime('1/1/2021 1:30 PM', '%m/%d/%Y %I:%M %p')
        d2 = datetime.strptime('10/1/2021 4:50 AM', '%m/%d/%Y %I:%M %p')
        data_h = random_date(d1, d2)
        insert = f"INSERT INTO rilevazione (tipo,data_ora,unita_misura,valore,codice_dispositivo) VALUES ('weight','{data_h}','grams','{row[1][1]}','{random.randint(1, 3)}');"
        f.write(insert)
    f.close()
    
create_rilevazione_txt(weight)

medici = pd.read_excel('medici.xlsx')

medici.head()

,NOME,COGNOME,EMAIL,TELEFONO,SPECIALIZZAZIONE
0,Sebastiano,Zaccagnini,dott.zaccagnini @med.com,3331234567,NaN
1,Marcella,Versace,dott.versace@med.com,3456698789,NaN
2,Giorgio,Tropea,dott.tropea@med.com,3249856889,NaN
3,Antonella,Fieramosca,dott.fieramosca@med.com,3250055593,NaN
4,Donato,Borromeo,dott.borromeo@med.com,3245599440,NaN


In [ ]:
# CREAZIONE MEDICI

def create_medici_txt(data):
    f = open("dml_insert.txt", "w")
    doc = 0
    for row in data.iterrows():
        doc += 1
        insert = f"INSERT INTO medico(nome,cognome,email,telefono,specializzazione) VALUES ('{row[1][0]}','{row[1][1]}','{row[1][2]}','{row[1][3]}','Ginecologo');"
        f.write(insert)
        
    f.close()
    return doc

doc = create_medici_txt(medici)

mamme = pd.read_excel('mamme.xlsx')

mamme.head()

,CODICE FISCALE,NOME,COGNOME,EMAIL,DATA NASCITA,TELEFONO,CODICE DISPOSITIVO
0,MVGXYF68M62C797V,Elena,Tamburi,elenatamburi@gmail.com,1987-02-12,7725331115,NaN
1,RSFLPH55A01Z153M,Rosalia,Bresciani,rosaliabresciani@gmail.com,1990-11-24,3370389484,NaN
2,MHBRJW62S49E015C,Camilla,Morabito,camillamorabito@gmail.com,1988-03-20,6071638199,NaN
3,GBAPSP92S12F937Q,Melania,Giacometti,melaniagiacometti@gmail.com,1977-02-24,5028680876,NaN
4,PPMRCR73B65C357P,Clelia,Bettoni,cleliabettoni@gmail.com,1995-04-27,4927386905,NaN


In [ ]:
# CREAZIONE MAMME

def create_mamme_txt(data):
    f = open("dml_insert.txt", "w")
    cont = 0
    mam = []
    for row in data.iterrows():
        cont += 1
        mam.append(row[1][0])
        insert = f"INSERT INTO mamma(cf,nome,cognome,email,data_nascita,telefono,codice_dispositivo) VALUES ('{row[1][0]}','{row[1][1]}','{row[1][2]}','{row[1][3]}','{row[1][4]}','{row[1][5]}',{cont});"
        f.write(insert)
            
    f.close()
    return mam


mam = create_mamme_txt(mamme)

In [ ]:
# CREAZIONE DISPOSITIVO, INCLUDE

def create_dispositivo_txt():
    f = open("dml_insert.txt", "w")
    #include = open("include.txt", "w")
    cont = 0
    letters = string.ascii_uppercase
    f.write(f"INSERT INTO tipo_sensore(unita_misura,tipo) VALUES('m/s^2','accellerometro');")
    f.write(f"INSERT INTO tipo_sensore(unita_misura,tipo) VALUES('gradi/s','giroscopio');")
    for row in range(1,51):
        cont += 1
        insert = f"INSERT INTO dispositivo(marca,modello,tipo,metodo) VALUES('{''.join(random.choice(letters) for i in range(10))}{cont}','{''.join(random.choice(letters) for i in range(10))}{cont}','{''.join(random.choice(letters) for i in range(10))}{cont}','{''.join(random.choice(letters) for i in range(10))}{cont}');"
        f.write(insert)
        insert2 = f"INSERT INTO include(codice_sensore,codice_dispositivo) VALUES('{random.randint(1, 2)}','{cont}');"
        f.write(insert2)
        
    f.close()
    #include.close()
    
create_dispositivo_txt()

In [ ]:
# CREAZIONE CONTROLLO

def create_controllo(number):
    f = open("dml_insert.txt","w")
    
    for i in range(1, number):
        month = random.randint(1, 12)
        s1 = f'{month}/1/2021 1:30 PM'
        s2 = f'{month}/20/2021 4:50 AM'
        d1 = datetime.strptime(s1, '%m/%d/%Y %I:%M %p')
        d2 = datetime.strptime(s2, '%m/%d/%Y %I:%M %p')
        data_h = random_date(d1, d2)
        mamma = random.choice(tuple(mam))
        insert = f"INSERT INTO controllo(codice_mamma,codice_medico,tipo,referto,data_ora) VALUES('{mamma}',{random.randint(1, doc)},'Controllo gravidanza','/mamma/{mamma}/{random.randint(1, doc)}.dat','{data_h}');"
        f.write(insert)
        
    f.close()

create_controllo(10000)

In [ ]:
# CREAZIONE SECONDA TRANCHE DATO

ddd = pd.read_excel('FetalPCGSpreadsheet.xlsx')
f = open("dml_insert.txt", "w")
ddd.iloc[:,7:8]

for row in ddd.iterrows():
    bpm = (row[1][7]).split('-')
    
    while("[]" in bpm) :
        bpm.remove("[]")
    test_list = [int(i) for i in bpm]
    mean = statistics.mean(test_list)
    d1 = datetime.strptime('1/1/2021 1:30 PM', '%m/%d/%Y %I:%M %p')
    d2 = datetime.strptime('10/1/2021 4:50 AM', '%m/%d/%Y %I:%M %p')
    data_h = random_date(d1, d2)

    insert = f"INSERT INTO dato (id,data_ora,tipo,descrizione,valore,nome,modalita) VALUES ({numero_dati+1},'{data_h}','CTG', 'Media valori fetal heart rate',"
    complete = insert+str(np.round(mean))+f",'bpm','{random.randint(1, 3)}');"
    numero_dati += 1
    f.write(complete)
    
f.close()

# Analytics

### 1) Quante visite una mamma effettua dal medico x nell'arco dell'anno

In [ ]:
connection,cursor = connect_db("db_dottore")

# medico codice = 5 
# mamma cf = MVGXYF68M62C797V

dataset = pd.read_sql_query("SELECT EXTRACT(months from data_ora) as mese,Count(*) as numero_visite, cf FROM (medico JOIN controllo ON medico.codice = controllo.codice_medico ) JOIN mamma ON controllo.codice_mamma = mamma.cf WHERE mamma.cf = 'MVGXYF68M62C797V' AND medico.codice = 5 GROUP BY mese, mamma.cf ORDER BY mese ",connection)

#dataset.plot(kind='bar',x='mese',y='numero_visite')

fig = go.Figure([go.Scatter(x=dataset['mese'], y=dataset['numero_visite'])])
fig.update_layout(legend_title_text = "Contestant")
fig.update_xaxes(title_text="Mese")
fig.update_yaxes(title_text="Numero visite effettuate")
fig.show()

PostgreSQL server information
You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 



### 2) La mamma che va più spesso dal medico x nell'arco dell'anno

In [ ]:
connection,cursor = connect_db("db_dottore")

# Nella sub query prendo il numero delle visite raggruppato per ciascun mese , per ogni mamma
# Nella query totale sommo il numero delle visite per ogni mamma e ordino in modo decrescente, prendendo il primo record
# corrispondente al massimo


dataset = pd.read_sql_query("""
                            SELECT SUM(numero_visite) as totale_visite, mamma.nome, mamma.cognome
                            FROM (SELECT EXTRACT(months from data_ora) as mese,Count(*) as numero_visite, cf FROM (medico JOIN controllo ON medico.codice = controllo.codice_medico ) JOIN mamma ON controllo.codice_mamma = mamma.cf 
                            WHERE medico.codice = 5 
                            GROUP BY mese, mamma.cf ORDER BY mese ) as T1 JOIN mamma on T1.cf=mamma.cf
                            GROUP BY mamma.cf
                            ORDER BY totale_visite DESC
                            LIMIT 1
                            
                            """,connection)

dataset


PostgreSQL server information
You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 



,totale_visite,nome,cognome
0,21.0,Amanda,Bonani


### 3) La mamma che va meno spesso dal medico x nell'arco dell'anno

In [ ]:
connection,cursor = connect_db("db_dottore")

# Nella sub query prendo il numero delle visite raggruppato per ciascun mese , per ogni mamma
# Nella query totale sommo il numero delle visite per ogni mamma e lascio l'ordine di default, prendendo il primo record
# corrispondente al minimo

dataset = pd.read_sql_query("""
                            SELECT SUM(numero_visite) as totale_visite, mamma.nome, mamma.cognome
                            FROM 
                            (SELECT EXTRACT(months from data_ora) as mese,Count(*) as numero_visite, cf FROM (medico JOIN controllo ON medico.codice = controllo.codice_medico ) JOIN mamma ON controllo.codice_mamma = mamma.cf 
                            WHERE medico.codice = 5
                            GROUP BY mese, mamma.cf ORDER BY mese ) as T1 JOIN mamma on T1.cf=mamma.cf
                            GROUP BY mamma.cf
                            ORDER BY totale_visite
                            LIMIT 1
                            
                            """,connection)

dataset

PostgreSQL server information
You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 



,totale_visite,nome,cognome
0,4.0,Carolina,Gussoni


### 4) Peso medio fetale rilevato

In [ ]:
connection,cursor = connect_db("db_dottore")


dataset = pd.read_sql_query("""
                            SELECT ROUND(AVG(cast(valore AS INTEGER))) as peso_medio, unita_misura
                            FROM rilevazione
                            WHERE tipo = 'weight'
                            GROUP BY unita_misura
                            """,connection)

dataset

PostgreSQL server information
You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 



,peso_medio,unita_misura
0,119.0,grams ...


### 5) Peso massimo fetale rilevato

In [ ]:
connection,cursor = connect_db("db_dottore")


dataset = pd.read_sql_query("""
                            SELECT MAX(cast(valore AS INTEGER)) as peso_medio, unita_misura
                            FROM rilevazione
                            WHERE tipo = 'weight'
                            GROUP BY unita_misura
                            """,connection)

dataset

PostgreSQL server information
You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 



,peso_medio,unita_misura
0,176,grams ...


### 6) Peso minimo fetale rilevato

In [ ]:
connection,cursor = connect_db("db_dottore")


dataset = pd.read_sql_query("""
                            SELECT MIN(cast(valore AS INTEGER)) as peso_medio, unita_misura
                            FROM rilevazione
                            WHERE tipo = 'weight'
                            GROUP BY unita_misura
                            """,connection)

dataset

PostgreSQL server information
You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 



,peso_medio,unita_misura
0,55,grams ...


### 7) Frequenza cardiaca media rilevata


In [ ]:
connection,cursor = connect_db("db_dottore")


ctg = pd.read_sql_query("""
                            SELECT MAX(valore) as massimo_valori, AVG(valore) as media_valori, MIN(valore) as minimo_valori
                            FROM dato
                            WHERE tipo = 'CTG'
                            """,connection)
ctg

PostgreSQL server information
You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 



,massimo_valori,media_valori,minimo_valori
0,172.0,141.045455,121.0


In [ ]:
connection,cursor = connect_db("db_dottore")


ctg = pd.read_sql_query("""
                            SELECT id, valore
                            FROM dato
                            WHERE tipo = 'CTG'
                            """,connection)

PostgreSQL server information
You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 



In [ ]:
fig = go.Figure([go.Scatter(y=ctg['valore'], x=ctg['id'])])

fig.update_layout(legend_title_text = "Contestant")
fig.update_xaxes(title_text="Rilevazione")
fig.update_yaxes(title_text="Fetal Heart Rate medio")

fig.show()

### 8) Tutti i controlli effettuati esclusivamente con il dispositivo personale

In [ ]:
connection,cursor = connect_db("db_mamma")


ctg = pd.read_sql_query("""
                         SELECT controllo.codice as codice_controllo, controllo.data_ora,controllo.referto, dispositivo.marca, dispositivo.modello, dispositivo.tipo
FROM controllo JOIN dispositivo ON controllo.codice_dispositivo = dispositivo.codice_dispositivo
WHERE controllo.tipo = 'Controllo con dispositivo'

                            """,connection)
ctg

PostgreSQL server information
You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 



,codice_controllo,data_ora,referto,marca,modello,tipo
0,4,2022-05-07 15:26:22,\test1\dispositivo\test3 ...,MGHBQCWVUJ1 ...,EATJMDHPLD1 ...,XBUWIJTERT1 ...
1,5,2022-05-07 15:26:22,\test1\dispositivo\test4 ...,MGHBQCWVUJ1 ...,EATJMDHPLD1 ...,XBUWIJTERT1 ...
2,6,2022-05-07 15:26:22,\test1\dispositivo\test5 ...,MGHBQCWVUJ1 ...,EATJMDHPLD1 ...,XBUWIJTERT1 ...


### 9) Tutti i controlli effettuati attraverso i dispositivi medici (non personali)

In [ ]:
connection,cursor = connect_db("db_mamma")


ctg = pd.read_sql_query("""
                         SELECT controllo.codice as codice_controllo, controllo.data_ora,controllo.referto, dispositivo.marca, dispositivo.modello, dispositivo.tipo
FROM controllo JOIN dispositivo ON controllo.codice_dispositivo = dispositivo.codice_dispositivo
WHERE controllo.tipo != 'Controllo con dispositivo'

                            """,connection)
ctg

PostgreSQL server information
You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 



,codice_controllo,data_ora,referto,marca,modello,tipo
0,7,2022-05-07 15:26:22,\test1\dispositivom\test6 ...,JHAFAIQSYB2 ...,AEFVCWHFLV2 ...,QUPWLFNYML2 ...
1,8,2022-05-07 15:26:22,\test1\dispositivom\test7 ...,JHAFAIQSYB2 ...,AEFVCWHFLV2 ...,QUPWLFNYML2 ...


### 10) Tutte le patologie rilevate dai controlli effettuati alle mamme

In [ ]:
connection,cursor = connect_db("db_mamma")


ctg = pd.read_sql_query("""
                         SELECT DISTINCT(nome), descrizione
                         FROM ((patologia JOIN relativo ON patologia.codice =relativo.codice_patologia) JOIN controllo ON controllo.codice = relativo.codice_controllo) 
                         WHERE patologia.codice = relativo.codice_patologia
                      """,connection)
ctg

PostgreSQL server information
You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 



,nome,descrizione
0,Candida,"La candidosi è un infezione micotica, quindi p..."
1,Depressione,"Stato danimo, temporaneo o persistente, caratt..."
2,Diabete in gravidanza,Il diabete è una malattia ad evoluzione progre...
3,Ipertensione in gravidanza,Con ipertensione gravidica o gestazionale si i...


In [ ]:
connection.close()